In [25]:
import nascar_api
from pathlib import Path

In [26]:
parent_dir = Path.cwd().parent
inator_dir = parent_dir.joinpath("collect_inator")
inator_dir.mkdir(exist_ok=True)

In [ ]:
confirm = input("THIS WILL OVERWRITE GENERATED ""_base"" FILES AND __init__ FILE.\nThis will NOT touch existing ""_logic"" files.\nContinue (Y/N)? ")

if confirm.lower() == "y":
   init_string = ""
   import_string = """#######################################################################
## THIS FILE IS AUTOMATICALLY GENERATED. DO NOT MAKE CHANGES HERE.
#######################################################################

from toolbox.nascar_api import Feeds, get_api_data
from sqlalchemy import Engine 
import pandas as pd
"""
   script_string = "\n\n"

   for fi, f in enumerate(nascar_api.Feeds):
      #print(fi, f.name, f.value)
      p = f.value.params
      param_string = (", " + ", ".join([f"{x}: int" for x in p])) if p != [] else ""
      dict_string = "{" + ", ".join([f"\"{x}\": {x}" for x in p]) + "}"
      pass_through_string = (", " + ", ".join([f"{x}={x}" for x in p])) if p != [] else ""
      comma_string = ", ".join(p)
      comma_string_with_quotes = ", ".join([f"\"{x}\"" for x in p])

      init_string += f"from ._collect_inator_base import get_{f.name.lower()}\n"
      
      import_string += f"from collect_inator import {f.name.lower()}_logic\n"

      script_string += f"""def get_{f.name.lower()}(url_header: str{param_string}, sql_engine: Engine | None = None, load_to_sql: bool = True) -> tuple[bool, dict[str, pd.DataFrame] | None]:
   api_result = get_api_data(Feeds.{f.name}, {dict_string}, url_header=url_header)

   if api_result["result"]:
      df_dict = {f.name.lower()}_logic.json_to_df(api_result["json"]{pass_through_string})
      
      if load_to_sql:
         if sql_engine == None: return (False, None)
         for name in df_dict:
            existing_df = pd.read_sql_table(table_name=name, con=sql_engine, schema="nascar")
            df = pd.concat([df_dict[name], existing_df]).drop_duplicates(ignore_index=True)
            df.to_sql(name, con=sql_engine, if_exists="replace", index=False)

      return (True, df_dict)
   else:
      print(f"API call failed for Feeds.{f.name}. URL response code = {{api_result["result_code"]}}")
      return (False, None)\n\n"""
      
      logic_dir = inator_dir.joinpath(f"{f.name.lower()}_logic").with_suffix(".py")
      if not logic_dir.exists():
         logic_dir.touch()
         logic_dir.write_text(f"""import pandas as pd

def json_to_df(json: dict | list | object{param_string}) -> dict[str, pd.DataFrame]:
   df = pd.DataFrame(json) # type: ignore
   # may not be needed, but a good reminder to check
   # df[[{comma_string_with_quotes}]] = [{comma_string}]
   return {{"{f.name.lower()}": df}}""")
         
   
   script_dir = inator_dir.joinpath("_collect_inator_base").with_suffix(".py")

   script_dir.touch()
   script_dir.write_text(import_string + script_string)
         
   init_dir = inator_dir.joinpath("__init__").with_suffix(".py")
   init_dir.write_text(init_string)

In [28]:
test = {"bob": 5, "steve": 7}

for x in test:
   print(x)

bob
steve
